In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from heapy.pipe.event import gbmTTE
from heapy.pipe.image import epImage
from heapy.util.time import fermi_utc_to_met, ep_utc_to_met

In [3]:
utc = '2024-02-19T06:21:44.15'
fermi_met = fermi_utc_to_met(utc)
ep_met = ep_utc_to_met(utc)

gbm_tte = gbmTTE.from_utc(utc=utc, det='n9')
gbm_tte.timezero = fermi_met
gbm_tte.filter_time([-200, 300])
gbm_tte.filter_energy([8, 900])
gbm_tte.lc_binsize = 2
gbm_lc_time = gbm_tte.lc_time
gbm_lc_net_rate = gbm_tte.lc_net_rate

wxt_event = epImage.from_wxtobs('ep13600003859wxtCMOS44l23v2', 's2')
wxt_event.timezero = ep_met
wxt_event.filter_time([-200, 465])
wxt_event.filter_pi([50, 400])
wxt_event.lc_binsize = 2
wxt_lc_time = wxt_event.lc_time
wxt_lc_net_rate = wxt_event.lc_net_rate

In [4]:
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = 'Georgia'
rcParams['font.size'] = 12
rcParams['pdf.fonttype'] = 42

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
p1, = ax.plot(gbm_lc_time, gbm_lc_net_rate, lw=1.0, c='r', label='Fermi/GBM')
ax.set_xlim([-100, 300])    #
ax.set_xlabel(f'Time since {utc} (s)')
ax.set_ylabel('Fermi/GBM counts per second')
ax.minorticks_on()
ax.tick_params(axis='x', which='both', direction='in', labelcolor='k', colors='k')
ax.tick_params(axis='y', which='both', direction='in', labelcolor='k', colors='k')
ax.tick_params(which='major', width=1.0, length=5)
ax.tick_params(which='minor', width=1.0, length=3)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

ax1 = ax.twinx()
p2, = ax1.plot(wxt_lc_time, wxt_lc_net_rate, lw=1.0, c='b', label='EP-WXT')
ax1.set_ylabel('EP-WXT counts per second')

ratio = np.max(gbm_lc_net_rate) / np.max(wxt_lc_net_rate)
gbm_ylim = [0.2 * np.min(gbm_lc_net_rate), 1.2 * np.max(gbm_lc_net_rate)]
wxt_ylim = [lim / ratio for lim in gbm_ylim]
ax.set_ylim(gbm_ylim)
ax1.set_ylim(wxt_ylim)

plt.legend(handles=[p1, p2], frameon=False)
fig.savefig('./curve.pdf', bbox_inches='tight', pad_inches=0.1, dpi=300)
plt.close(fig)